# Question Answering Task

In [1]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
import logging

Here I read in the data and check the format that it's stored in. I want to make sure to transform the data into the format that simpletransformers takes.

In [2]:
train_data = pd.read_json("./qna/train.json")
eval_data = pd.read_json("./qna/test.json")

train_data

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...
...,...,...
437,v2.0,"{'title': 'Infection', 'paragraphs': [{'qas': ..."
438,v2.0,"{'title': 'Hunting', 'paragraphs': [{'qas': [{..."
439,v2.0,"{'title': 'Kathmandu', 'paragraphs': [{'qas': ..."
440,v2.0,"{'title': 'Myocardial_infarction', 'paragraphs..."


In [3]:
t_data = []
for row in train_data.index:
    t_data.extend(train_data.iloc[row, 1]["paragraphs"])

e_data = []
for row in eval_data.index:
    e_data.extend(eval_data.iloc[row, 1]["paragraphs"])

In [4]:
t_data

[{'qas': [{'question': 'When did Beyonce start becoming popular?',
    'id': '56be85543aeaaa14008c9063',
    'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
    'is_impossible': False},
   {'question': 'What areas did Beyonce compete in when she was growing up?',
    'id': '56be85543aeaaa14008c9065',
    'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
    'is_impossible': False},
   {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
    'id': '56be85543aeaaa14008c9066',
    'answers': [{'text': '2003', 'answer_start': 526}],
    'is_impossible': False},
   {'question': 'In what city and state did Beyonce  grow up? ',
    'id': '56bf6b0f3aeaaa14008c9601',
    'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
    'is_impossible': False},
   {'question': 'In which decade did Beyonce become famous?',
    'id': '56bf6b0f3aeaaa14008c9602',
    'answers': [{'text': 'late 1990s', 'answer_start': 276}],
    'is_imposs

I chose the RoBERTa model to train on. 

In [5]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True

model = QuestionAnsweringModel(
    "roberta", "roberta-base", args=model_args, cuda_device=6
)

# Train the model
model.train_model(t_data, eval_data=e_data)

# Evaluate the model
result, texts = model.eval_model(e_data)

# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

answers, probabilities = model.predict(to_predict)

print(answers)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/8215 [00:01<?, ?it/s]

/storage/de30373/anaconda3/envs/nlp/lib/python3.8/site-packages/simpletransformers/question_answering/question_answering_model.py:751: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/storage/de30373/anaconda3/envs/nlp/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `

Running Evaluation:   0%|          | 0/1512 [00:00<?, ?it/s]

...

INFO:simpletransformers.question_answering.question_answering_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 11873/11873 [00:00<00:00, 271206.68it/s]


Running Evaluation:   0%|          | 0/1512 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 17119.61it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['great power and skill.', 'power and skill.', 'a Mistborn of great power and skill.', 'Mistborn of great power and skill.', 'great power', 'Vin is a Mistborn of great power and skill.', 'power', '', 'skill.', 'a Mistborn of great power', 'Mistborn of great power', 'a Mistborn', 'Mistborn', '.', 'Vin is a Mistborn of great power', 'of great power and skill.', 'Vin is a Mistborn', 'is a Mistborn of great power and skill.', 'and skill.']}]


I decided to ask it a question that was a bot more tricky, one that required more contextual/grammatical understanding. I was throroughly impressed.

In [6]:
to_predict = [
    {
        "context": "A few years after the First Crusade, in 1107, the Normans under the command of Bohemond, Robert's son, landed in Valona and besieged Dyrrachium using the most sophisticated military equipment of the time, but to no avail. Meanwhile, they occupied Petrela, the citadel of Mili at the banks of the river Deabolis, Gllavenica (Ballsh), Kanina and Jericho. This time, the Albanians sided with the Normans, dissatisfied by the heavy taxes the Byzantines had imposed upon them. With their help, the Normans secured the Arbanon passes and opened their way to Dibra. The lack of supplies, disease and Byzantine resistance forced Bohemond to retreat from his campaign and sign a peace treaty with the Byzantines in the city of Deabolis.",
        "qas": [
            {
                "question": 'What city did the Normans occupy?',
                "id": "0",
            }
        ],
    }
]

model.predict(to_predict)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 7182.03it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0',
   'answer': ['Petrela,',
    'Dyrrachium using the most sophisticated military equipment of the time, but to no avail. Meanwhile, they occupied Petrela,',
    'Petrela, the citadel of Mili at the banks of the river Deabolis, Gllavenica (Ballsh), Kanina and Jericho.',
    'Valona and besieged Dyrrachium using the most sophisticated military equipment of the time, but to no avail. Meanwhile, they occupied Petrela,',
    'Petrela, the citadel of Mili at the banks of the river Deabolis, Gllavenica (Ballsh), Kanina',
    'Petrela, the citadel of Mili',
    'Dyrrachium',
    'Dyrrachium using the most sophisticated military equipment of the time, but to no avail. Meanwhile, they occupied Petrela, the citadel of Mili at the banks of the river Deabolis, Gllavenica (Ballsh), Kanina and Jericho.',
    'Valona and besieged Dyrrachium',
    'Valona and besieged Dyrrachium using the most sophisticated military equipment of the time, but to no avail. Meanwhile, they occupied Petrela, 

I chose a random paragraph from the evaluation dataset to make my question from.

In [7]:
e_data[12]

{'qas': [{'question': 'Where did the Normans and Byzantines sign the peace treaty?',
   'id': '56de15104396321400ee25b7',
   'answers': [{'text': 'Deabolis', 'answer_start': 302},
    {'text': 'Deabolis', 'answer_start': 718},
    {'text': 'Deabolis', 'answer_start': 718}],
   'is_impossible': False},
  {'question': "Who was Robert's son?",
   'id': '56de15104396321400ee25b8',
   'answers': [{'text': 'Bohemond', 'answer_start': 79},
    {'text': 'Bohemond', 'answer_start': 79},
    {'text': 'Bohemond', 'answer_start': 79}],
   'is_impossible': False},
  {'question': 'What river was Petrela located by?',
   'id': '56de15104396321400ee25b9',
   'answers': [{'text': 'Deabolis', 'answer_start': 302},
    {'text': 'the river Deabolis', 'answer_start': 292},
    {'text': 'Deabolis', 'answer_start': 302}],
   'is_impossible': False},
  {'plausible_answers': [{'text': 'Dyrrachium', 'answer_start': 133}],
   'question': 'Who did the Normans besiege in the 11th century?',
   'id': '5ad3ee2d604f3